# Gdal tutorial

### Open and close a raster dataset

In [1]:
import gdal

# open dataset
ds = gdal.Open('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/dtm/25-46.tif')

# close dataset
ds = None

### Get Raster Metadata

In [2]:
# open dataset
gtif = gdal.Open('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/dtm/25-46.tif')

In [3]:
print(gtif.GetMetadata())

{'AREA_OR_POINT': 'Area'}


### Get Raster Band

In [4]:
from osgeo import gdal
import sys
# this allows GDAL to throw Python Exceptions
gdal.UseExceptions()

try:
    src_ds = gdal.Open('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/dtm/25-46.tif')
except RuntimeError:
    print('Unable to open INPUT.tif')
    #print(e)
    sys.exit(1)

try:
    srcband = src_ds.GetRasterBand(1)
except RuntimeError:
    # for example, try GetRasterBand(10)
    print('Band ( %i ) not found' % band_num)
    #print(e)
    sys.exit(1)


In [5]:
srcband.GetStatistics(True, True)

RuntimeError: Failed to compute statistics, no valid pixels found in sampling.

### Loop Through All Raster Bands

In [6]:
from osgeo import gdal
import sys

src_ds = gdal.Open('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/oh/oh.tif')
if src_ds is None:
    print('Unable to open INPUT.tif')
    sys.exit(1)

print("[ RASTER BAND COUNT ]: ", src_ds.RasterCount)
for band in range( src_ds.RasterCount ):
    band += 1
    print("[ GETTING BAND ]: ", band)
    srcband = src_ds.GetRasterBand(band)
    if srcband is None:
        continue

    stats = srcband.GetStatistics( True, True )
    if stats is None:
        continue

    print("[ STATS ] =  Minimum=%.3f, Maximum=%.3f, Mean=%.3f, StdDev=%.3f" % ( \
                stats[0], stats[1], stats[2], stats[3] ))

[ RASTER BAND COUNT ]:  1
[ GETTING BAND ]:  1
[ STATS ] =  Minimum=-340282306073709992518750102204469739520.000, Maximum=45.000, Mean=-186529541738350012835662197320696463360.000, StdDev=169350030033880009086770168648261697536.000


In [7]:
srcband.GetStatistics( True, True )

[-3.4028230607371e+38, 45.0, -1.8652954173835e+38, 1.6935003003388e+38]

In [11]:
src_ds.GetGeoTransform()

(373758.8101525671,
 0.9999999999999799,
 0.0,
 418695.6451956801,
 0.0,
 -0.99999999999998)

### Get extend

In [31]:
#src = gdal.Open('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/oh/oh.tif')
#src = gdal.Open('/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/tmp/raster_1_temp.tif')
src = gdal.Open('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/dtm/25-46.tif')
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
lrx = ulx + (src.RasterXSize * xres)
lry = uly + (src.RasterYSize * yres)

In [32]:
print('upper left corner: ', ulx, uly)
print('lower right corner: ', lrx, lry)

upper left corner:  173462.08030918773 389578.5985227945
lower right corner:  175027.55069879314 387762.213018567


In [36]:
from osgeo import gdal
import sys
gdal.UseExceptions()

def Usage():
    print("""
    $ getrasterband.py [ band number ] input-raster
    """)
    sys.exit(1)

def main( band_num, input_file ):
    src_ds = gdal.Open( input_file )
    if src_ds is None:
        print('Unable to open %s' % input_file)
        sys.exit(1)

    try:
        srcband = src_ds.GetRasterBand(band_num)
    except RuntimeError:
        print('No band %i found' % band_num)
        sys.exit(1)


    print("[ NO DATA VALUE ] = ", srcband.GetNoDataValue())
    print("[ MIN ] = ", srcband.GetMinimum())
    print("[ MAX ] = ", srcband.GetMaximum())
    print("[ SCALE ] = ", srcband.GetScale())
    #print("[ UNIT TYPE ] = ", srcband.GetUnitType())
    ctable = srcband.GetColorTable()

    if ctable is None:
        print('No ColorTable found')
        sys.exit(1)

    print("[ COLOR TABLE COUNT ] = ", ctable.GetCount())
    for i in range( 0, ctable.GetCount() ):
        entry = ctable.GetColorEntry( i )
        if not entry:
            continue
        print("[ COLOR ENTRY RGB ] = ", ctable.GetColorEntryAsRGB( i, entry ))

if __name__ == '__main__':

    if len( sys.argv ) < 3:
        print("""
        [ ERROR ] you must supply at least two arguments:
        1) the band number to retrieve and 2) input raster
        """)
        Usage()

    main( 1, '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/tmp/raster_1_temp.tif' )

[ NO DATA VALUE ] =  None
[ MIN ] =  None
[ MAX ] =  None
[ SCALE ] =  None
No ColorTable found


SystemExit: 1

In [18]:
### http://karthur.org/2015/clipping-rasters-in-python.html

In [19]:
from osgeo import gdal, gdalnumeric, ogr
from PIL import Image, ImageDraw
import os
import numpy as np

def clip_raster(rast, features_path, gt=None, nodata=-9999):
    '''
    Clips a raster (given as either a gdal.Dataset or as a numpy.array
    instance) to a polygon layer provided by a Shapefile (or other vector
    layer). If a numpy.array is given, a "GeoTransform" must be provided
    (via dataset.GetGeoTransform() in GDAL). Returns an array. Clip features
    must be a dissolved, single-part geometry (not multi-part). Modified from:

    http://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html
    #clip-a-geotiff-with-shapefile

    Arguments:
        rast            A gdal.Dataset or a NumPy array
        features_path   The path to the clipping features
        gt              An optional GDAL GeoTransform to use instead
        nodata          The NoData value; defaults to -9999.
    '''
    def array_to_image(a):
        '''
        Converts a gdalnumeric array to a Python Imaging Library (PIL) Image.
        '''
        i = Image.fromstring('L',(a.shape[1], a.shape[0]),
            (a.astype('b')).tostring())
        return i

    def image_to_array(i):
        '''
        Converts a Python Imaging Library (PIL) array to a gdalnumeric image.
        '''
        a = gdalnumeric.fromstring(i.tobytes(), 'b')
        a.shape = i.im.size[1], i.im.size[0]
        return a

    def world_to_pixel(geo_matrix, x, y):
        '''
        Uses a gdal geomatrix (gdal.GetGeoTransform()) to calculate
        the pixel location of a geospatial coordinate; from:
        http://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html#clip-a-geotiff-with-shapefile
        '''
        ulX = geo_matrix[0]
        ulY = geo_matrix[3]
        xDist = geo_matrix[1]
        yDist = geo_matrix[5]
        rtnX = geo_matrix[2]
        rtnY = geo_matrix[4]
        pixel = int((x - ulX) / xDist)
        line = int((ulY - y) / xDist)
        return (pixel, line)

    # Can accept either a gdal.Dataset or numpy.array instance
    if not isinstance(rast, np.ndarray):
        gt = rast.GetGeoTransform()
        rast = rast.ReadAsArray()

    # Create an OGR layer from a boundary shapefile
    features = ogr.Open(features_path)
    if features.GetDriver().GetName() == 'ESRI Shapefile':
        lyr = features.GetLayer(os.path.split(os.path.splitext(features_path)[0])[1])

    else:
        lyr = features.GetLayer()

    # Get the first feature
    poly = lyr.GetNextFeature()

    # Convert the layer extent to image pixel coordinates
    minX, maxX, minY, maxY = lyr.GetExtent()
    ulX, ulY = world_to_pixel(gt, minX, maxY)
    lrX, lrY = world_to_pixel(gt, maxX, minY)

    # Calculate the pixel size of the new image
    pxWidth = int(lrX - ulX)
    pxHeight = int(lrY - ulY)

    # If the clipping features extend out-of-bounds and ABOVE the raster...
    if gt[3] < maxY:
        # In such a case... ulY ends up being negative--can't have that!
        iY = ulY
        ulY = 0

    # Multi-band image?
    try:
        clip = rast[:, ulY:lrY, ulX:lrX]

    except IndexError:
        clip = rast[ulY:lrY, ulX:lrX]

    # Create a new geomatrix for the image
    gt2 = list(gt)
    gt2[0] = minX
    gt2[3] = maxY

    # Map points to pixels for drawing the boundary on a blank 8-bit,
    #   black and white, mask image.
    points = []
    pixels = []
    geom = poly.GetGeometryRef()
    pts = geom.GetGeometryRef(0)

    for p in range(pts.GetPointCount()):
        points.append((pts.GetX(p), pts.GetY(p)))

    for p in points:
        pixels.append(world_to_pixel(gt2, p[0], p[1]))

    raster_poly = Image.new('L', (pxWidth, pxHeight), 1)
    rasterize = ImageDraw.Draw(raster_poly)
    rasterize.polygon(pixels, 0) # Fill with zeroes

    # If the clipping features extend out-of-bounds and ABOVE the raster...
    if gt[3] < maxY:
        # The clip features were "pushed down" to match the bounds of the
        #   raster; this step "pulls" them back up
        premask = image_to_array(raster_poly)
        # We slice out the piece of our clip features that are "off the map"
        mask = np.ndarray((premask.shape[-2] - abs(iY), premask.shape[-1]), premask.dtype)
        mask[:] = premask[abs(iY):, :]
        mask.resize(premask.shape) # Then fill in from the bottom

        # Most importantly, push the clipped piece down
        gt2[3] = maxY - (maxY - gt[3])

    else:
        mask = image_to_array(raster_poly)

    # Clip the image using the mask
    try:
        clip = gdalnumeric.choose(mask, (clip, nodata))

    # If the clipping features extend out-of-bounds and BELOW the raster...
    except ValueError:
        # We have to cut the clipping features to the raster!
        rshp = list(mask.shape)
        if mask.shape[-2] != clip.shape[-2]:
            rshp[0] = clip.shape[-2]

        if mask.shape[-1] != clip.shape[-1]:
            rshp[1] = clip.shape[-1]

        mask.resize(*rshp, refcheck=False)

        clip = gdalnumeric.choose(mask, (clip, nodata))

    return (clip, ulX, ulY, gt2)


ModuleNotFoundError: No module named 'PIL'

In [ ]:
### Create a Polygon

In [20]:
from osgeo import ogr

# Create ring
ring = ogr.Geometry(ogr.wkbLinearRing)
ring.AddPoint(1179091.1646903288, 712782.8838459781)
ring.AddPoint(1161053.0218226474, 667456.2684348812)
ring.AddPoint(1214704.933941905, 641092.8288590391)
ring.AddPoint(1228580.428455506, 682719.3123998424)
ring.AddPoint(1218405.0658121984, 721108.1805541387)
ring.AddPoint(1179091.1646903288, 712782.8838459781)

# Create polygon
poly = ogr.Geometry(ogr.wkbPolygon)
poly.AddGeometry(ring)

print(poly.ExportToWkt())

POLYGON ((1179091.16469033 712782.883845978 0,1161053.02182265 667456.268434881 0,1214704.9339419 641092.828859039 0,1228580.42845551 682719.312399842 0,1218405.0658122 721108.180554139 0,1179091.16469033 712782.883845978 0))


In [ ]:
### Create a MultiPolygon

In [ ]:
from osgeo import ogr

multipolygon = ogr.Geometry(ogr.wkbMultiPolygon)

# Create ring #1
ring1 = ogr.Geometry(ogr.wkbLinearRing)
ring1.AddPoint(1204067.0548148106, 634617.5980860253)
ring1.AddPoint(1204067.0548148106, 620742.1035724243)
ring1.AddPoint(1215167.4504256917, 620742.1035724243)
ring1.AddPoint(1215167.4504256917, 634617.5980860253)
ring1.AddPoint(1204067.0548148106, 634617.5980860253)

# Create polygon #1
poly1 = ogr.Geometry(ogr.wkbPolygon)
poly1.AddGeometry(ring1)
multipolygon.AddGeometry(poly1)

# Create ring #2
ring2 = ogr.Geometry(ogr.wkbLinearRing)
ring2.AddPoint(1179553.6811741155, 647105.5431482664)
ring2.AddPoint(1179553.6811741155, 626292.3013778647)
ring2.AddPoint(1194354.20865529, 626292.3013778647)
ring2.AddPoint(1194354.20865529, 647105.5431482664)
ring2.AddPoint(1179553.6811741155, 647105.5431482664)

# Create polygon #2
poly2 = ogr.Geometry(ogr.wkbPolygon)
poly2.AddGeometry(ring2)
multipolygon.AddGeometry(poly2)

print(multipolygon.ExportToWkt())

In [21]:
poly

<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x7f262162c2a0> >

# create polygon and save it un a ESRI shapefile

In [23]:
#src = gdal.Open('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/oh/oh.tif')
src = gdal.Open('/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/tmp/raster_1_temp.tif')
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
lrx = ulx + (src.RasterXSize * xres)
lry = uly + (src.RasterYSize * yres)

print('upper left corner: ', ulx, uly)
print('lower right corner: ', lrx, lry)

upper left corner:  373759.8101529634 418694.64519406826
lower right corner:  451231.8101521982 381707.6451944336


In [24]:
from osgeo import ogr

# Create ring
ring = ogr.Geometry(ogr.wkbLinearRing)
ring.AddPoint(ulx, uly)
ring.AddPoint(ulx, lry)
ring.AddPoint(lrx, lry)
ring.AddPoint(lrx, uly)

# Create polygon
poly = ogr.Geometry(ogr.wkbPolygon)
poly.AddGeometry(ring)

print(poly.ExportToWkt())

POLYGON ((373759.810152963 418694.645194068 0,373759.810152963 381707.645194434 0,451231.810152198 381707.645194434 0,451231.810152198 418694.645194068 0))


In [29]:
import osgeo.ogr as ogr
import osgeo.osr as osr

# set up the shapefile driver
driver = ogr.GetDriverByName("ESRI Shapefile")

# create the data source
data_source = driver.CreateDataSource("test.shp")

# create the spatial reference, WGS84
srs = osr.SpatialReference()
srs.ImportFromEPSG(31287)

# create the layer
layer = data_source.CreateLayer("poly", srs, ogr.wkbPolygon)

#field_region = ogr.FieldDefn("Region", ogr.OFTString)
#field_region.SetWidth(24)
#layer.CreateField(ogr.FieldDefn("Latitude", ogr.OFTReal))
#layer.CreateField(ogr.FieldDefn("Longitude", ogr.OFTReal))
layer.CreateField(ogr.FieldDefn('id', ogr.OFTInteger))
defn = layer.GetLayerDefn()

# Create a new feature (attribute and geometry)
feature = ogr.Feature(defn)
feature.SetField('id', 123)

# Make a geometry, from Shapely object
geom = ogr.CreateGeometryFromWkb(poly.ExportToWkt())
feature.SetGeometry(geom)

# Create the feature in the layer (shapefile)
layer.CreateFeature(feature)
# Dereference the feature
feature = None

# Save and close the data source
data_source = None